In [66]:
import json
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient # kismi sonradan ekle

In [59]:
ws = Workspace.from_config()

In [67]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

pipeline_cluster = "yk-compute-cluster"
compute_target = ws.compute_targets[pipeline_cluster]

In [41]:
data = pd.read_csv(
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
)
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [68]:
from azureml.data.dataset_factory import TabularDatasetFactory

datastore = ws.get_default_datastore()#we will store intermediate data between data preparation and automated ml step in the workspace default datastore.
train_data = TabularDatasetFactory.register_pandas_dataframe(
    data, target=(datastore, "dataset/"), name="AutoMLE2EPipeline_Classification_train"
)
#I creat dataset from pandas dataframe
#Other way: You can create  dataset by using ds=Dataset.Tabular.from_delimited_files(path=xxx) and register it with ds.register()

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to dataset//64608141-42eb-43d6-bc8c-1389844f9d68/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [119]:
#Target: Configure the training run
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

#env = Environment.get(workspace=ws, name='experiment_env', version='2')

pipeline_run_config=RunConfiguration()
pipeline_run_config.target= compute_target

# Add conda dependencies to the automl env:
pipeline_run_config.environment.python.conda_dependencies = CondaDependencies.create(
 conda_packages=['pandas'])  

In [120]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
import datetime

#1st way: OutputFileDatasetConfig without giving destination
#prepped_output_path = OutputFileDatasetConfig(name="output_path") 

#2nd way: OutputFileDatasetConfig with giving destination
output_path = (datastore, f"azureml/classification_prep_output/")
prepped_output_path = OutputFileDatasetConfig(destination = output_path)
# OutputFileDatasetConfig  points a directory and CSV file will be written there. Also given as parameter below.
# If you do not give destination parameter, it will copy the output to the workspaceblobstore datastore, under the path /dataset/{run-id}/{output-name}. 
# Run-id is the name value on the overview of the job and outputname is the name given as a parameter. In my example, it is output_path.

input_ds = Dataset.get_by_name(ws, 'AutoMLE2EPipeline_Classification_train')

prep_step=PythonScriptStep(
    name="Prepare AutoML Classification",
    script_name="prepare.py",
    source_directory="./Scripts",
    #arguments=['--input-data',input_ds.as_named_input('AutoMLE2EPipeline_Classification_train'),'--prepped-data',output],
    arguments=["--output_path",prepped_output_path],
    inputs=[input_ds.as_named_input('AutoMLE2EPipeline_Classification_train')],
    compute_target=compute_target,
    runconfig=pipeline_run_config
)

In [121]:
#In an ML pipeline, the input data must be a Dataset object.

prepped_data=prepped_output_path.read_delimited_files() # This is the data to send automlstep

from azureml.pipeline.core import TrainingOutput,PipelineData

metrics_data = PipelineData(name='metrics_data',
                            datastore=datastore,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='best_model_data',
                          datastore=datastore,
                          pipeline_output_name='model_output',
                          training_output=TrainingOutput(type='Model'))
# two pipelinedata objects for automl outputs: metrics and the model.


In [122]:
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 2,
    "experiment_timeout_hours" : 0.25,
    "primary_metric" : 'AUC_weighted'
}
#the run will stop after only 2 iterations or 15 minutes, whichever comes first.

automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target= compute_target,
                             run_configuration = pipeline_run_config,
                             featurization = 'auto',
                             training_data = prepped_data,
                             label_column_name = 'y',
                             **automl_settings)
# debug_log local file if you want to see logs

train_step = AutoMLStep(name='AutoML_Classification',
    automl_config=automl_config,
    outputs=[metrics_data,model_data],
    enable_default_model_output=False,
    enable_default_metrics_output=False,)

In [126]:
from azureml.pipeline.core.graph import PipelineParameter

# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="AutoML_Classification_Test")

register_step = PythonScriptStep(
     name="register_model",
     script_name="register_model.py",
     source_directory="./Scripts",
     allow_reuse=False,
     arguments=["--model_name", model_name, "--model_path", model_data],
     inputs=[model_data],
     compute_target=compute_target,
     runconfig=pipeline_run_config)

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

experiment = Experiment(ws, name= "automl-classification-E2E_trainingPipeline")

pipeline = Pipeline(ws, [prep_step, train_step,register_step])

run = experiment.submit(pipeline, show_output=True)
run.wait_for_completion()